In [1]:
import os
import json
import torch
import cv2 as cv
import numpy as np
from PIL import Image
import torch.nn as nn
from tqdm import tqdm
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

# Step 3
mainFolderPath = './Data-Inference/'
modelPath = r'D:\Teknofest\Inference\Inference\4x-ConvNext-SWIN-E1-448.pt'

#Read Folders
folderPaths = [mainFolderPath + f for f in os.listdir(mainFolderPath)]
pIDs = [str(f)  for f in os.listdir(mainFolderPath)]

print("Length of the folders: ", len(folderPaths))
print("Length of the pIDs: ", len(pIDs))


Length of the folders:  800
Length of the pIDs:  800


In [2]:
#Define The Object
JSONoutputs = {
    "kunye": {
        "takim_adi": "PIXELERS",
        "takim_id": "539822",
        "aciklama": "",
        "versiyon": "0.0"
    },
    "tahminler": {
        
	}
}

In [3]:
import matplotlib.pyplot as plt


class ConvNext(nn.Module):
	def __init__(self, num_classes=4):
		super(ConvNext, self).__init__()
		
		self.convnext = models.convnext_small(pretrained=False)
		self.convnext2 = models.convnext_small(pretrained=False)
		self.convnext3 = models.convnext_small(pretrained=False)
		self.convnext4 = models.convnext_small(pretrained=False)

		# Replace the classifiers with identities to extract features
		self.convnext.classifier[2] = nn.Identity()
		self.convnext2.classifier[2] = nn.Identity()
		self.convnext3.classifier[2] = nn.Identity()
		self.convnext4.classifier[2] = nn.Identity()
		
		self.fc = nn.Sequential(
			nn.Linear(3072  , 512),
			nn.Dropout(0.2),
			nn.ReLU(),
			nn.Linear(512, num_classes)
		)

	def forward(self, image,image2):
		B, C, H, W = image.size()
		
		if H % 2 != 0 or W % 2 != 0:
			raise ValueError("Image height and width must be divisible by 2.")
		
		top_left = image[:, :, :H//2, :W//2]
		top_right = image[:, :, :H//2, W//2:]
		bottom_left = image[:, :, H//2:, :W//2]
		bottom_right = image[:, :, H//2:, W//2:]
		
		features1 = self.convnext(top_left)
		features2 = self.convnext2(top_right)
		features3 = self.convnext3(bottom_left)
		features4 = self.convnext4(bottom_right)
		
		combined_features = torch.cat((features1, features2, features3, features4), dim=1)
		
		output = self.fc(combined_features.view(combined_features.size(0), -1))
		
		return output



class SWIN(nn.Module):
	def __init__(self, num_classes=4):
		super(SWIN, self).__init__()
		
		self.swin = models.swin_b(pretrained=False)
		self.swin2 = models.swin_b(pretrained=False)
		self.swin3 = models.swin_b(pretrained=False)
		self.swin4 = models.swin_b(pretrained=False)

		# Replace the classifiers with identities to extract features
		self.swin.head = nn.Identity()
		self.swin2.head = nn.Identity()
		self.swin3.head = nn.Identity()
		self.swin4.head = nn.Identity()
		
		self.fc = nn.Sequential(
			nn.Linear(4096  , 512),
			nn.Dropout(0.2),
			nn.ReLU(),
			nn.Linear(512, num_classes)
		)

	def forward(self, image,image2):
		B, C, H, W = image.size()
		
		if H % 2 != 0 or W % 2 != 0:
			raise ValueError("Image height and width must be divisible by 2.")
		
		top_left = image[:, :, :H//2, :W//2]
		top_right = image[:, :, :H//2, W//2:]
		bottom_left = image[:, :, H//2:, :W//2]
		bottom_right = image[:, :, H//2:, W//2:]
		
		features1 = self.swin(top_left)
		features2 = self.swin2(top_right)
		features3 = self.swin3(bottom_left)
		features4 = self.swin4(bottom_right)
		
		combined_features = torch.cat((features1, features2, features3, features4), dim=1)
		
		output = self.fc(combined_features.view(combined_features.size(0), -1))
		
		return output
	
 
	
class CombinedModel(nn.Module):
	def __init__(self, num_classes=4):
		super(CombinedModel, self).__init__()
		
		self.convNext = ConvNext()
		self.swin = SWIN()
		# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
		
		# self.convNext.load_state_dict(torch.load('./Saved/4x-ConvNext-E10-448.pt', map_location=device))
		# self.swin.load_state_dict(torch.load('./Saved/4x-SWIN-E16-448.pt', map_location=device))
		
		# Replace the classifiers with identities to extract features
		self.convNext.fc = nn.Identity()
		self.swin.fc = nn.Identity()


		self.fc = nn.Sequential(
            nn.Linear(7168, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )

	def forward(self, image1):
		features1 = self.convNext(image1, '')
		features3 = self.swin(image1, '')
		combined_features = torch.cat((features1,features3), dim=1)		
		
		output = self.fc(combined_features)
		
		return output

transform = transforms.Compose([
	transforms.Resize((448, 448)),
	transforms.ToTensor(),
	transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])

class CustomImageDataset(Dataset):
	def __init__(self, transform=None):
		self.transform = transform
		self.image_data = []

		for i in tqdm(range(len(folderPaths))):
			image_dir = folderPaths[i]
			pID = pIDs[i]

			files = os.listdir(image_dir)
			if 'CC' in files[0]:
				file1 = os.path.join(image_dir, files[0])
				file2 = os.path.join(image_dir, files[1])
			else:
				file1 = os.path.join(image_dir, files[1])
				file2 = os.path.join(image_dir, files[0])

			self.image_data.append((file1,file2, pID))
	
	def __len__(self):
		return len(self.image_data)
	
	def __getitem__(self, idx):
		file1, file2, pID = self.image_data[idx]
		
		image1 = cv.imread(file1)
		image2 = cv.imread(file2)

		
		combined_image = cv.hconcat([image1, image2])
		
		combined_image = Image.fromarray(combined_image)
		if self.transform:
			newImage = self.transform(combined_image)

		return newImage, pID
	
def get_batches(dataset, batch_size, shuffle=False):
	indices = np.arange(len(dataset))
	for start_idx in range(0, len(dataset), batch_size):
		batch_indices = indices[start_idx:start_idx + batch_size]
		batch = [dataset[idx] for idx in batch_indices]
		images, pIDS = zip(*batch)
		images = torch.stack(images)
		yield images, pIDS

In [4]:
data = CustomImageDataset(transform=transform)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = CombinedModel(num_classes=4)
model.to(device)

# Load the model weights
model.load_state_dict(torch.load(modelPath, map_location=device))

model.eval()
all_preds = []
all_pIDS = []

with torch.no_grad():
	for images, pIDS in get_batches(data, 8):
		images = images.to(device)
		outputs = model(images)
		_, preds = torch.max(outputs, 1)
		all_preds.extend(preds.cpu().numpy())
		all_pIDS.extend(pIDS)
		print(len(all_preds))

print("Length of the predictions: ", len(all_preds))
print("Length of the pIDs: ", len(all_pIDS))



100%|██████████| 800/800 [00:00<00:00, 18172.70it/s]
c:\Users\ihirc\anaconda3\envs\U_Net\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ihirc\anaconda3\envs\U_Net\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


cuda
8
16
24
32
40
48
56
64
72
80
88
96
104
112
120
128
136
144
152
160
168
176
184
192
200
208
216
224
232
240
248
256
264
272
280
288
296
304
312
320
328
336
344
352
360
368
376
384
392
400
408
416
424
432
440
448
456
464
472
480
488
496
504
512
520
528
536
544
552
560
568
576
584
592
600
608
616
624
632
640
648
656
664
672
680
688
696
704
712
720
728
736
744
752
760
768
776
784
792
800
Length of the predictions:  800
Length of the pIDs:  800


In [5]:
for i in range(len(all_pIDS)):
	cat = '1'
	if (all_preds[i] == 1):
		cat = '2'
	elif (all_preds[i] == 2):
		cat = '4'
	elif (all_preds[i] == 3):
		cat = '5'
	JSONoutputs['tahminler'][all_pIDS[i]] = {
		"kategori" : cat
	}


In [6]:
with open ('outputs.json', 'w') as f:
	json.dump(JSONoutputs, f, indent=4)